In [20]:
import numpy as np
import matplotlib as plt
import qrcode
import warnings
import cv2

In [15]:
n=np.array(
    [
        [0,255,0,255,0,255],
        [255,0,255,0,255,0],
        [0,255,0,255,0,255],
        [255,0,255,0,255,0],
        [0,255,0,255,0,255],
        [255,0,255,0,255,0]
    ]
)

In [16]:
n.shape

(6, 6)

In [17]:
n_f = n.flatten()
n_f

array([  0, 255,   0, 255,   0, 255, 255,   0, 255,   0, 255,   0,   0,
       255,   0, 255,   0, 255, 255,   0, 255,   0, 255,   0,   0, 255,
         0, 255,   0, 255, 255,   0, 255,   0, 255,   0])

In [18]:
def encode(image, file_name='compressed_image.txt', bits=15):

    count_list = []
    count = 0
    prev = None
    fimage = image.flatten()
    size = 2**(bits+1) - 2**bits #32768 = 8000H
    
    for pixel in fimage:
        
        if prev == None:
            prev = pixel 
            count += 1
        else:
            if prev != pixel: # The current pixel is different from the previous one
                if count >= 3:
                    count_list.append((size+count, [prev]))
                else:
                    if count_list == []:
                        count_list.append((count, [prev]*count))
                    else: 
                        c, color = count_list[-1]
                        if c > size: 
                            count_list.append((count, [prev]*count))
                        else:
                            if c+count <= (2**bits)-1: # Make sure that we didn't use all of the 15 bits reserved for the color's sequence length 
                                count_list[-1] = (c+count, color+[prev]*count)
                            else:
                                count_list.append((count, [prev]*count))
                prev = pixel
                count = 1
            else: # The current pixel is like the previous one
                if count < (2**bits)-1: # Make sure that we didn't use all of the 15 bits reserved for the number of repetitions 
                    count += 1                    
                else:
                    count_list.append((size+count, [prev]))
                    prev = pixel
                    count = 1

    if count >= 3:
        count_list.append((size+count, [prev]))
    else:
        c, color = count_list[-1]
        if c > size:
            count_list.append((count, [prev]*count))
        else:
            if c+count <= (2**bits)-1:
                count_list[-1] = (c+count, color+[prev]*count)
            else:
                count_list.append((count, [prev]*count))

    # Hexa encoding
    with open(file_name,"w") as file:
        hexa_encoded = "".join(map(lambda x: "{0:04x}".format(x[0])+"".join(map(lambda y: "{0:02x}".format(y), x[1])), count_list))
        """ hexa_encoded = ""
        for count, colors in count_list:
            hexa_encoded += "{0:04x}".format(count)
            for color in colors:
                hexa_encoded += "{0:02x}".format(color) """
        file.write(hexa_encoded)
        
    # Compression rate
    rate = (1-(len(hexa_encoded)/2)/len(fimage))*100
    
    return hexa_encoded, rate

In [19]:
encode(n)

('002400ff00ff00ffff00ff00ff0000ff00ff00ffff00ff00ff0000ff00ff00ffff00ff00ff00',
 -5.555555555555558)

In [21]:
image1_np = np.array([[0, 1, 0, 1, 0, 1], 
                    [1, 0, 1, 0, 1, 0],
                    [0, 1, 0, 1, 0, 1],
                    [1, 0, 1, 0, 1, 0], 
                    [0, 1, 0, 1, 0, 1],
                    [1, 0, 1, 0, 1, 0]])*255

image2_np = np.array([[0, 0, 0, 1, 1, 1], 
                    [1, 1, 1, 0, 0, 0],
                    [0, 0, 0, 1, 1, 1],
                    [0, 0, 0, 1, 1, 1], 
                    [1, 1, 1, 0, 0, 0],
                    [0, 0, 0, 1, 1, 1]])*255

image3_np = np.array([[0, 0, 0, 0, 0, 0], 
                    [0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1], 
                    [0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0]])*255

In [37]:
code , rate = encode(image2_np)
print(code, rate)

8003008006ff8006008003ff8003008006ff8006008003ff 33.333333333333336


In [39]:
def decode(coded_image):
    i=0
    decoded_image = []
    while (i<len(coded_image)):
        binary_value = str(bin(int(coded_image[i], 16)))[2:].zfill(16)
        if(binary_value[0] == "1"):
            values_to_add = int(binary_value[1:], 2)
            decoded_image = decoded_image +  [coded_image[i+1]] * values_to_add
            i=i+2
        else:
            counter = int(binary_value, 2)
            decoded_image = decoded_image + [value for value in coded_image[i+1:i+counter+1]]
            i =  counter + 1
    return np.array(decoded_image)